[![img/pythonista.png](img/pythonista.png)](https://www.pythonista.io)

## La extensión *Flask-Marshmallow*.

https://flask-marshmallow.readthedocs.io

In [1]:
!pip install flask-marshmallow

You should consider upgrading via the '/home/oi/pythonista/bin/python -m pip install --upgrade pip' command.


In [9]:
from flask import Flask, abort, request
from flask_restful import Api, Resource
from flask_sqlalchemy import SQLAlchemy
from flask_marshmallow import Marshmallow, base_fields
from marshmallow.validate import Length, Range, OneOf
from data import carreras
from json import loads

In [10]:
app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///data/alumnosql.sqlite"
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False
# Order matters: Initialize SQLAlchemy before Marshmallow
api = Api(app)
db = SQLAlchemy(app)
ma = Marshmallow(app)

In [11]:
class Alumno(db.Model):
    __tablename__ = 'alumno'
    cuenta = db.Column(db.Integer, primary_key=True)
    nombre = db.Column(db.String(50))
    primer_apellido = db.Column(db.String(50))
    segundo_apellido = db.Column(db.String(50))
    carrera = db.Column(db.String(50))
    semestre = db.Column(db.Integer)
    promedio = db.Column(db.Float)
    al_corriente = db.Column(db.Boolean)

In [12]:
class AlumnoSchema(ma.SQLAlchemySchema):
    
    
    class Meta:
        model = Alumno
        load_instance = True
    
    
    cuenta = base_fields.Int(required=True, validate=Range(min=1000000, max=9999999))
    nombre = base_fields.String(required=True, validate=Length(min=2, max=50))
    primer_apellido = base_fields.String(required=True, validate=Length(min=2, max=50))
    segundo_apellido = base_fields.String(required=False, validate=Length(min=2, max=50))
    carrera = base_fields.String(required=True, validate=OneOf(carreras))
    semestre = base_fields.Int(required=True, validate=Range(min=1, max=50))
    promedio = base_fields.Float(required=True, validate=Range(min=1, max=50))
    al_corriente = base_fields.Bool(required=True)

In [13]:
schema_multiple = AlumnoSchema(many=True)
schema = AlumnoSchema()

In [14]:
class MuestraAlumnos(Resource):
    
    
    def get(Resource):
        alumnos = Alumno.query.all()
        return schema_multiple.dump(alumnos)

In [15]:
class ABCAlumno(Resource):
    
    
    def get(self, cuenta):
        alumno = Alumno.query.filter_by(cuenta=cuenta).first_or_404()
        return schema.dump(alumno)
    
    
    def delete(self, cuenta):
        alumno = Alumno.query.filter_by(cuenta=cuenta).first_or_404()
        db.session.delete(alumno)
        db.session.commit()
        return schema.dump(alumno)
     
    
    def post(self, cuenta):
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno:
            abort(409)
        else:
            respuesta = loads(request.data)
            respuesta["cuenta"] = cuenta
            db.session.add(schema.load(respuesta))
            db.session.commit()
            return respuesta
     
    
    def put(self, cuenta):
        alumno = Alumno.query.filter_by(cuenta=cuenta).first_or_404()
        respuesta = loads(request.data)
        respuesta["cuenta"] = cuenta
        db.session.add(schema.load(respuesta))
        db.session.commit()
        return respuesta

In [16]:
api.add_resource(ABCAlumno, '/api/<string:cuenta>')
api.add_resource(MuestraAlumnos, '/api/')

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://10.0.2.15:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Feb/2022 21:32:51] "GET /api/ HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2022 21:32:55] "GET /api/1231221 HTTP/1.1" 404 -
[2022-02-17 21:33:36,152] ERROR in app: Exception on /api/1231268 [POST]
Traceback (most recent call last):
  File "/home/oi/pythonista/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/oi/pythonista/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/home/oi/pythonista/lib/python3.9/site-packages/flask_restful/__init__.py", line 467, in wrapper
    resp = resource(*args, **kwargs)
  File "/home/oi/pythonista/lib/python3.9/site-packages/flask/views.py", line 84, in view
    return current_app.ensure_sync(self.dispatch_request)(*args, **kwargs)
  File "/home/oi/pythonista/lib/python

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2022.</p>